In [0]:
#Reading the csv file
filepath = "dbfs:/FileStore/Global_Retail/Bronze_Layer/Customer/customer.csv"

df = spark.read.csv(filepath, header = True, inferSchema = True)

In [0]:
df.show()

+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+
|customer_id|       name|               email|  country|customer_type|registration_date|age|gender|total_purchases|
+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+
|          1| Customer 1|customer1@example...|Australia|      Regular|       2011-05-15| 22|  Male|            191|
|          2| Customer 2|customer2@example...|   France|      Premium|       2018-11-27| 52| Other|            145|
|          3| Customer 3|customer3@example...|   Canada|      Premium|       2015-10-01| 32| Other|            691|
|          4| Customer 4|customer4@example...|      USA|      Premium|       2011-01-19| 70| Other|            644|
|          5| Customer 5|customer5@example...|  Germany|      Regular|       2021-08-26| 66| Other|            508|
|          6| Customer 6|customer6@example...|   France|      Premium|  

In [0]:
df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- country: string (nullable = true)
 |-- customer_type: string (nullable = true)
 |-- registration_date: date (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- total_purchases: integer (nullable = true)



In [0]:
from pyspark.sql.functions import current_timestamp
new_df = df.withColumn("ingestion_Timestamp",current_timestamp())

In [0]:
new_df.show()

+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+--------------------+
|customer_id|       name|               email|  country|customer_type|registration_date|age|gender|total_purchases| ingestion_Timestamp|
+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+--------------------+
|          1| Customer 1|customer1@example...|Australia|      Regular|       2011-05-15| 22|  Male|            191|2024-10-08 22:58:...|
|          2| Customer 2|customer2@example...|   France|      Premium|       2018-11-27| 52| Other|            145|2024-10-08 22:58:...|
|          3| Customer 3|customer3@example...|   Canada|      Premium|       2015-10-01| 32| Other|            691|2024-10-08 22:58:...|
|          4| Customer 4|customer4@example...|      USA|      Premium|       2011-01-19| 70| Other|            644|2024-10-08 22:58:...|
|          5| Customer 5|customer5@exampl

In [0]:
#saving as a Table now in Delta Table

spark.sql("use globalretail_bronze")

new_df.write \
.format('delta') \
.mode('append') \
.saveAsTable('bronze_customer')

In [0]:
spark.sql("select * from bronze_customer limit 10").show()

+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+--------------------+
|customer_id|       name|               email|  country|customer_type|registration_date|age|gender|total_purchases| ingestion_Timestamp|
+-----------+-----------+--------------------+---------+-------------+-----------------+---+------+---------------+--------------------+
|          1| Customer 1|customer1@example...|Australia|      Regular|       2011-05-15| 22|  Male|            191|2024-10-08 23:01:...|
|          2| Customer 2|customer2@example...|   France|      Premium|       2018-11-27| 52| Other|            145|2024-10-08 23:01:...|
|          3| Customer 3|customer3@example...|   Canada|      Premium|       2015-10-01| 32| Other|            691|2024-10-08 23:01:...|
|          4| Customer 4|customer4@example...|      USA|      Premium|       2011-01-19| 70| Other|            644|2024-10-08 23:01:...|
|          5| Customer 5|customer5@exampl

In [0]:
import datetime
archive_folder = "dbfs:/FileStore/GlobalRetail/bronze_layer/customer_data/archive/"
archive_filepath = archive_folder +'_'+ datetime.datetime.now().strftime("%Y%m%d%H%M%s")
print(archive_filepath)

dbfs:/FileStore/GlobalRetail/bronze_layer/customer_data/archive/_2024100823281728430112


In [0]:
dbutils.fs.mv(filepath,archive_filepath)

---------------------------------------------------------------------------
UnsupportedOperationException             Traceback (most recent call last)
File <command-1387934050476904>, line 1
----> 1 dbutils.fs.mv(filepath,archive_filepath)

File /databricks/python_shell/dbruntime/dbutils.py:364, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    362 def f_with_exception_handling(*args, **kwargs):
    363     try:
--> 364         return f(*args, **kwargs)
    365     except Py4JJavaError as e:
    367         class ExecutionError(Exception):

File /databricks/python_shell/dbruntime/dbutils.py:419, in DBUtils.FSHandler.mv(self, source, dest, recurse)
    417     return self.print_return(self.fsutils_parallel.mvParallel(source, dest))
    418 else:
--> 419     return self.print_return(self.fsutils.mv(source, dest, recurse))

File /databricks/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py:1355, in JavaMember.__call__(self, 

In [0]:
%sql

show databases